We will implement Perceptron for a binary classification task — bank-note authen- tication. Please download the data “bank-note.zip” from Canvas. The features and labels are listed in the file “bank-note/data-desc.txt”. The training data are stored in the file “bank-note/train.csv”, consisting of 872 examples. The test data are stored in “bank-note/test.csv”, and comprise of 500 examples. In both the training and testing datasets, feature values and labels are separated by commas.

Implement the standard Perceptron. Set the maximum number of epochs T to 10. Report your learned weight vector, and the average prediction error on the test dataset.

In [38]:
import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
%matplotlib inline
seed = np.random.seed

In [39]:
test = pd.read_csv('/content/sample_data/test.csv')
train = pd.read_csv('/content/sample_data/train.csv')
test = test.values
train = train.values

In [40]:
print(test[0:9,:])

[[ 2.8521    9.171    -3.6461   -1.2047    0.      ]
 [ 5.2418   10.5388   -4.1174   -4.2797    0.      ]
 [-2.2623   12.1177    0.28846  -7.7581    0.      ]
 [ 0.55298  -3.4619    1.7048    1.1008    1.      ]
 [ 4.1542    7.2756   -2.4766   -1.2099    0.      ]
 [-1.7279   -6.841     8.9494    0.68058   1.      ]
 [-1.4454   -8.4385    8.8483    0.96894   1.      ]
 [ 4.5459    8.1674   -2.4586   -1.4621    0.      ]
 [-0.048008 -0.56078   7.7215    0.453     0.      ]]


In [41]:
train_x = np.array(train[:, :-1])
test_x = np.array(test[:, :-1])
train_y = np.array(train[:, -1])
test_y = np.array(test[:, -1])
train_y = train_y.astype('int')
test_y = test_y.astype('int')

In [42]:
print(test_y[0:9])

[0 0 0 1 0 1 1 0 0]


In [43]:
train_y[train_y==0] = -1
test_y[test_y==0] = -1
print(test_y[0:9])

[-1 -1 -1  1 -1  1  1 -1 -1]


In [8]:
#set initial seed
s = 1

In [28]:
class Perceptron(object):
    def __init__(self, r=0.01, T=10, seed=s):
        self.r = r
        self.T = T
        self.seed = seed
    
    def FinalInput(self, X):
        return np.dot(X, self.w[1:]) + self.w[0]
            
    def Fit(self, X, y):
        rnd = np.random.RandomState(self.seed)
        self.w = rnd.normal(loc=0.0, scale=0.01, size=1+X.shape[1])
        self.errors_list = []
        for epoch in range(self.T):
            errors = 0.0
            for i in range(len(y)):
                xi = X[i]
                yi = y[i]
                output = self.Predict(xi)
                error = (yi - output)
                self.w[1:] += self.r * error * xi
                self.w[0] += self.r * error
                errors += int(error != 0.0)
            self.errors_list.append(errors)
        return self
    
    def Predict(self, X):
        return np.where(self.FinalInput(X) >= 0.0, 1, -1)

In [29]:
E = 0
n = 20
for i in range(n):
    p = Perceptron(r=0.01, T=10, seed=s)
    p.Fit(train_x, train_y)
    predicton = p.Predict(test_x)
    print(p.w)
    print('Prediction error: %d' % (test_y != predicton).sum())
    E += (test_y != predicton).sum()
    s = int(np.random.rand()*n*10)

print('Average prediction error: ', E / n)

[ 1.02496714 -1.18213144 -0.89556831 -0.85497493 -0.09236527]
Prediction error: 9
[ 1.08750722 -1.32198411 -0.82404387 -0.82467923 -0.21460653]
Prediction error: 7
[ 1.03987675 -1.20161003 -0.86141931 -0.92469211 -0.0676128 ]
Prediction error: 6
[ 1.01290498 -1.21825644 -0.75258518 -0.83122749 -0.20264361]
Prediction error: 7
[ 1.06969685 -1.2689439  -0.84397007 -0.75162169 -0.19679622]
Prediction error: 9
[ 1.02264163 -1.2834524  -0.77569395 -0.74660963 -0.09265707]
Prediction error: 10
[ 1.03899853 -1.24361775 -0.82273389 -0.89759269 -0.07182081]
Prediction error: 6
[ 1.06026375 -1.26168776 -0.80464386 -0.88018802 -0.15594227]
Prediction error: 7
[ 1.08288479 -1.25974438 -0.80819444 -0.87078476 -0.15101412]
Prediction error: 6
[ 1.03055524 -1.21900454 -0.80943253 -0.82469208 -0.16066952]
Prediction error: 7
[ 1.05668068 -1.28464636 -0.83629323 -0.90023161 -0.12464666]
Prediction error: 7
[ 1.11329212 -1.28178231 -0.925531   -0.82037542 -0.22443992]
Prediction error: 8
[ 1.03868388 -1

Implement the voted Perceptron. Set the maximum number of epochs T to 10. Report the list of the distinct weight vectors and their counts — the number of correctly predicted training examples. Using this set of weight vectors to predict each test example. Report the average test error.

In [75]:
class VotedPerceptron(object):
    def __init__(self, r=0.01, T=10, seed=1):
        self.r = r
        self.T = T
        self.seed = seed

    def FinalInput(self, X):
        return np.dot(X, self.w[1:]) + self.w[0]
    def Fit(self, X, y):
        rnd = np.random.RandomState(self.seed)
        self.w = rnd.normal(loc=0.0, scale=0.01, size=1+X.shape[1])
        self.errors_list = []
        for epoch in range(self.T):
            errors = 0.0
            for i in range(len(y)):
                xi = X[i]
                yi = y[i]
                output = self.Predict(xi)
                error = (yi - output)
                self.w[1:] += self.r * error * xi
                self.w[0] += self.r * error
                errors += int(error != 0.0)
            self.errors_list.append(errors)
        return self
    def Fit(self, X, y):
        rnd = np.random.RandomState(self.seed)
        self.w = rnd.normal(loc=0.0, scale=0.0, size=1+X.shape[1])
        self.c = 0
        self.errors_list = []
        self.List = []
        for m in range(self.T):
            errors = 0.0
            for i in range(len(y)):
                xi = X[i]
                yi = y[i]
                output = self.Predict(xi)
                error = (yi - output)
            if error != 0.0:
                self.w[m+1,1:] = self.w[m,1:] + self.r * error * xi
                self.w[m+1,0] = self.w[m,0] + self.r * error
                self.c = 1
            else:
                self.c += 1
            errors += int(error != 0.0)
            self.List[m,:-1] = self.w[m,:]
            self.List[m,-1] = self.c
            self.errors_list.append(errors)
        return self
    
    def Predict(self, X):
        return np.where(self.FinalInput(X) >= 0.0, 1, -1)

In [67]:
    p = Perceptron(r=0.01, T=10, seed=1)
    p.Fit(train_x, train_y)
    predicton = p.Predict(test_x)
    print(p.List)
    print('Average prediction error: %d' % (test_y != predicton).sum())

AttributeError: ignored

Implement the average Perceptron. Set the maximum number of epochs T to 10. Report your learned weight vector. Comparing with the list of weight vectors from (b), what can you observe? Report the average prediction error on the test data.

Compare the average prediction errors for the three methods. What do you conclude?

In [44]:
print(1+train_x.shape[1])

5
